<a href="https://colab.research.google.com/github/GiuBonacchi/Music/blob/main/sistema_de_reconhecimento_musical_util.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Colocar dataframe com nome novos_dados.csv

In [50]:
import pickle
import pandas as pd
import numpy as np

In [51]:
with open('modelo_random_forest.pkl', 'rb') as f:
    model = pickle.load(f)
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)
with open('colunas_usadas_no_treino.pkl', 'rb') as f:
    colunas_treino = pickle.load(f)
print('carregados com sucesso')

carregados com sucesso


In [52]:
numeric_cols = ['duration_ms', 'music_popularity', 'artist_popularity', 'followers']

In [53]:
novos_dados = pd.read_csv('novos_dados.csv')

In [54]:
nome_musicas = novos_dados['music'].copy()

In [55]:
# para eliminar colunas extras e colunas com dados faltantes
for col in colunas_treino:
    if col not in novos_dados.columns:
        novos_dados[col] = 0

In [56]:
novos_dados = novos_dados[colunas_treino]

In [57]:
novos_dados['music'] = nome_musicas.values

In [58]:
novos_dados[numeric_cols] = scaler.transform(novos_dados[numeric_cols])

In [59]:
#criado pq nas colunas de treino não usei a coluna music
novos_dados_a = novos_dados[colunas_treino]

In [60]:
previsao = model.predict(novos_dados_a)

In [61]:
print(previsao)

[1 0 0 0 0 1 0 0 0 0 0 1 0 1 1 0 0 1 0 0 1 1 0 1 0 0 0 0 1 0 0 0 1 0 1 0 1
 0 1 0 1 0 1 0 0 0 0 1 0 1]


In [62]:
print(np.sum(previsao))

18


In [63]:
novos_dados['liked_previsao'] = previsao
print(novos_dados[['music','liked_previsao']].head(20))

      music  liked_previsao
0    Song 1               1
1    Song 2               0
2    Song 3               0
3    Song 4               0
4    Song 5               0
5    Song 6               1
6    Song 7               0
7    Song 8               0
8    Song 9               0
9   Song 10               0
10  Song 11               0
11  Song 12               1
12  Song 13               0
13  Song 14               1
14  Song 15               1
15  Song 16               0
16  Song 17               0
17  Song 18               1
18  Song 19               0
19  Song 20               0
